## 事件流处理器

In [1]:
from illufly.io import EventBlock
EventBlock("Chunk", "hi")

EventBlock(block_type=<chunk>, content=<hi>)

### 实现流交换

基于大模型的AI应用中经常要求模型流式返回，我们有一种实现流输出的标准实现。

In [2]:
from illufly.types import Runnable

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield "hi\n"
        yield "illufly!\n"

r = MyRun()
r()

hi
illufly!


In [3]:
r.events_history.store

{'20241129-67663-0000-0376': {'threads': {},
  'callings': {'20241129-67663-0000-4660': [{'id': '20241129-67663-0002-3594',
     'event': 'message',
     'data': '{"block_id": "20241129-67663-0002-3594", "block_type": "runnable", "content": "MyRun.4568773488", "content_id": "20241129-67663-0003-9023", "created_at": "2024-11-29T16:07:43.601380", "thread_id": null, "calling_id": "20241129-67663-0000-4660", "agent_name": "MyRun.4568773488", "model_name": null}'},
    {'id': '20241129-67663-0004-9015',
     'event': 'message',
     'data': '{"block_id": "20241129-67663-0004-9015", "block_type": "text", "content": "hi\\n", "content_id": "20241129-67663-0005-9224", "created_at": "2024-11-29T16:07:43.601449", "thread_id": null, "calling_id": "20241129-67663-0000-4660", "agent_name": "MyRun.4568773488", "model_name": null}'},
    {'id': '20241129-67663-0006-5237',
     'event': 'message',
     'data': '{"block_id": "20241129-67663-0006-5237", "block_type": "text", "content": "illufly!\\n", "co

### 推送事件流

In [4]:
for x in r(generator="sync", verbose=False):
    print("SSE >>", x)

SSE >> {'id': '20241129-67723-0012-7849', 'event': 'message', 'data': '{"block_id": "20241129-67723-0012-7849", "block_type": "text", "content": "hi\\n", "content_id": "20241129-67723-0013-8387", "created_at": "2024-11-29T16:08:43.427174", "thread_id": null, "calling_id": "20241129-67723-0001-5231", "agent_name": "MyRun.4568773488", "model_name": null}'}
SSE >> {'id': '20241129-67723-0016-0648', 'event': 'message', 'data': '{"block_id": "20241129-67723-0016-0648", "block_type": "text", "content": "illufly!\\n", "content_id": "20241129-67723-0017-2515", "created_at": "2024-11-29T16:08:43.427482", "thread_id": null, "calling_id": "20241129-67723-0001-5231", "agent_name": "MyRun.4568773488", "model_name": null}'}


In [5]:
async for x in r(generator="async", verbose=False):
    print("SSE >>", x)

SSE >> {'id': '20241129-67724-0022-0058', 'event': 'message', 'data': '{"block_id": "20241129-67724-0022-0058", "block_type": "text", "content": "hi\\n", "content_id": "20241129-67724-0023-6530", "created_at": "2024-11-29T16:08:44.028100", "thread_id": null, "calling_id": "20241129-67724-0002-3152", "agent_name": "MyRun.4568773488", "model_name": null}'}
SSE >> {'id': '20241129-67724-0026-1436', 'event': 'message', 'data': '{"block_id": "20241129-67724-0026-1436", "block_type": "text", "content": "illufly!\\n", "content_id": "20241129-67724-0027-9011", "created_at": "2024-11-29T16:08:44.028664", "thread_id": null, "calling_id": "20241129-67724-0002-3152", "agent_name": "MyRun.4568773488", "model_name": null}'}


In [6]:
r.events_history.store

{'20241129-67663-0000-0376': {'threads': {},
  'callings': {'20241129-67663-0000-4660': [{'id': '20241129-67663-0002-3594',
     'event': 'message',
     'data': '{"block_id": "20241129-67663-0002-3594", "block_type": "runnable", "content": "MyRun.4568773488", "content_id": "20241129-67663-0003-9023", "created_at": "2024-11-29T16:07:43.601380", "thread_id": null, "calling_id": "20241129-67663-0000-4660", "agent_name": "MyRun.4568773488", "model_name": null}'},
    {'id': '20241129-67663-0004-9015',
     'event': 'message',
     'data': '{"block_id": "20241129-67663-0004-9015", "block_type": "text", "content": "hi\\n", "content_id": "20241129-67663-0005-9224", "created_at": "2024-11-29T16:07:43.601449", "thread_id": null, "calling_id": "20241129-67663-0000-4660", "agent_name": "MyRun.4568773488", "model_name": null}'},
    {'id': '20241129-67663-0006-5237',
     'event': 'message',
     'data': '{"block_id": "20241129-67663-0006-5237", "block_type": "text", "content": "illufly!\\n", "co

In [7]:
from illufly.types import Runnable

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield ["hi\n", "illufly!\n"]

r = MyRun()
r()

['hi\n', 'illufly!\n']

### Runnable 事件流

In [8]:
from illufly.types import Runnable, EventBlock

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield EventBlock("chunk", "hi, ")
        yield EventBlock("chunk", "illufly")
        yield EventBlock("chunk", "!")

def handle(block, **kwargs):
    print(block.json)

r = MyRun()
r("hi")

hi, illufly!

In [9]:
r.events_history.store

{'20241129-67739-0002-7970': {'threads': {},
  'callings': {'20241129-67739-0004-2541': [{'id': '20241129-67739-0032-2494',
     'event': 'message',
     'data': '{"block_id": "20241129-67739-0032-2494", "block_type": "runnable", "content": "MyRun.4706565568", "content_id": "20241129-67739-0033-9221", "created_at": "2024-11-29T16:08:59.094146", "thread_id": null, "calling_id": "20241129-67739-0004-2541", "agent_name": "MyRun.4706565568", "model_name": null}'},
    {'id': '20241129-67739-0034-4463',
     'event': 'message',
     'data': '{"block_id": "20241129-67739-0034-4463", "block_type": "chunk", "content": "hi, ", "content_id": "20241129-67739-0035-2321", "created_at": "2024-11-29T16:08:59.094241", "thread_id": null, "calling_id": "20241129-67739-0004-2541", "agent_name": null, "model_name": null}'},
    {'id': '20241129-67739-0036-7953',
     'event': 'message',
     'data': '{"block_id": "20241129-67739-0036-7953", "block_type": "chunk", "content": "illufly", "content_id": "20241

### ChatAgent 事件流

In [10]:
from illufly.types import Runnable, EventBlock
from illufly.chat import FakeLLM
from illufly.io import log, alog

def my_handler(block, **kwargs):
    print("[", block.block_type, "]", block.text)

r = FakeLLM(handlers=[my_handler], sleep=0.5)
r("hi")

[ runnable ] FakeLLM.4568762544
[ human ] hi
[ agent ] FakeLLM.4568762544
[ info ] 记住 10 轮对话
[ info ] I am FakeLLM
[ chunk ] R
[ chunk ] e
[ chunk ] p
[ chunk ] l
[ chunk ] y
[ chunk ]  
[ chunk ] >
[ chunk ] >
[ chunk ]  
[ chunk ] h
[ chunk ] i
[ new_line ] 
[ final_text ] Reply >> hi


'Reply >> hi'

In [11]:
from illufly.types import Runnable, EventBlock
from illufly.chat import FakeLLM
from illufly.io import log, alog

r = FakeLLM(handlers=[alog], sleep=0.1)
await r("hi")

[HUMAN] hi
[AGENT] FakeLLM.4706565520
Reply >> hi


'Reply >> hi'

In [12]:
r.events_history.store

{'20241129-67758-0004-7641': {'threads': {'FakeLLM.4706565520': '20241129-67758-0001-7056'},
  'callings': {'20241129-67758-0006-8941': [{'id': '20241129-67758-0075-5306',
     'event': 'message',
     'data': '{"block_id": "20241129-67758-0075-5306", "block_type": "runnable", "content": "FakeLLM.4706565520", "content_id": "20241129-67758-0076-4983", "created_at": "2024-11-29T16:09:18.700119", "thread_id": null, "calling_id": "20241129-67758-0006-8941", "agent_name": "FakeLLM.4706565520", "model_name": null}'},
    {'id': '20241129-67758-0077-3636',
     'event': 'message',
     'data': '{"block_id": "20241129-67758-0077-3636", "block_type": "human", "content": "hi", "content_id": "20241129-67758-0078-1555", "created_at": "2024-11-29T16:09:18.701752", "thread_id": null, "calling_id": "20241129-67758-0006-8941", "agent_name": "FakeLLM.4706565520", "model_name": null}'},
    {'id': '20241129-67758-0079-1297',
     'event': 'message',
     'data': '{"block_id": "20241129-67758-0079-1297",

### 推送 ChatAgent 事件流

In [12]:
from illufly.chat import FakeLLM

llm = FakeLLM(name="if1", handlers=[])
llm.create_new_thread()

async for b in llm("给我写一篇200字小作文", generator="async", sleep=0.1):
    print(b)

{'id': '20241129-45454-0178-6510', 'event': 'message', 'data': '{"block_id": "20241129-45454-0178-6510", "block_type": "runnable", "content": "if1", "content_id": "20241129-45454-0179-9139", "created_at": "2024-11-29T09:57:34.705933", "thread_id": "20241129-45454-0004-8782", "calling_id": "20241129-45454-0009-0319", "agent_name": "if1", "model_name": null}'}
{'id': '20241129-45454-0180-3694', 'event': 'message', 'data': '{"block_id": "20241129-45454-0180-3694", "block_type": "user", "content": "给我写一篇200字小作文", "content_id": "20241129-45454-0181-2420", "created_at": "2024-11-29T09:57:34.706812", "thread_id": "20241129-45454-0004-8782", "calling_id": "20241129-45454-0009-0319", "agent_name": "if1", "model_name": null}'}
{'id': '20241129-45454-0182-5094', 'event': 'message', 'data': '{"block_id": "20241129-45454-0182-5094", "block_type": "agent", "content": "if1", "content_id": "20241129-45454-0183-9378", "created_at": "2024-11-29T09:57:34.707047", "thread_id": "20241129-45454-0004-8782", 

In [13]:
async for b in llm("给我写一篇200字小作文", generator="async", sleep=0.1):
    print(b)

{'id': '20241129-45476-0233-4101', 'event': 'message', 'data': '{"block_id": "20241129-45476-0233-4101", "block_type": "runnable", "content": "if1", "content_id": "20241129-45476-0234-1992", "created_at": "2024-11-29T09:57:56.725106", "thread_id": "20241129-45454-0004-8782", "calling_id": "20241129-45476-0010-9033", "agent_name": "if1", "model_name": null}'}
{'id': '20241129-45476-0235-5575', 'event': 'message', 'data': '{"block_id": "20241129-45476-0235-5575", "block_type": "user", "content": "给我写一篇200字小作文", "content_id": "20241129-45476-0236-6834", "created_at": "2024-11-29T09:57:56.725721", "thread_id": "20241129-45454-0004-8782", "calling_id": "20241129-45476-0010-9033", "agent_name": "if1", "model_name": null}'}
{'id': '20241129-45476-0237-6512', 'event': 'message', 'data': '{"block_id": "20241129-45476-0237-6512", "block_type": "agent", "content": "if1", "content_id": "20241129-45476-0238-3258", "created_at": "2024-11-29T09:57:56.725955", "thread_id": "20241129-45454-0004-8782", 

## 嵌套 BaseAgent

In [1]:
from illufly.types import BaseAgent, PromptTemplate
from illufly.chat import ChatQwen
from illufly.flow import ReAct

def get_city(location: str):
    """由任意地名或地址描述查询出所在的城市"""
    return "广州"

def get_weather(city: str):
    """我可以查询城市的天气情况。city必须是城市名称。"""
    return f'{city}天气不错，一直是大晴天'

def booking(request: str):
    """我可以根据你的需求预订球场"""
    return '我已经帮你预订好了，祝你玩得愉快'

store = {}
flow = ReAct(
    ChatQwen(name="react-planner", tools=[get_city, get_weather, booking])
)

for b in flow("我在鸿蒙公司，帮我查询一下天气情况", verbose=False, generator="sync"):
    print(b)

{'event': 'log', 'id': '20241122-89486-0000-5272', 'data': {'content': 'ReAct.4557867328', 'block_type': 'runnable', 'content_id': '20241122-89486-0001-8567', 'created_at': '2024-11-22T23:31:26.754816', 'thread_id': None, 'calling_id': '20241122-89486-0000-8696', 'agent_name': 'ReAct.4557867328', 'model_name': None}}
{'event': 'log', 'id': '20241122-89486-0002-7206', 'data': {'content': '>>> flow-node-1: planner', 'block_type': 'node', 'content_id': '20241122-89486-0003-8546', 'created_at': '2024-11-22T23:31:26.755321', 'thread_id': None, 'calling_id': '20241122-89486-0000-8696', 'agent_name': 'ReAct.4557867328', 'model_name': None}}
{'event': 'log', 'id': '20241122-89486-0004-3324', 'data': {'content': '我在鸿蒙公司，帮我查询一下天气情况', 'block_type': 'user', 'content_id': '20241122-89486-0005-9716', 'created_at': '2024-11-22T23:31:26.755581', 'thread_id': None, 'calling_id': '20241122-89486-0000-8696', 'agent_name': 'react-planner', 'model_name': 'qwen-plus'}}
[AGENT] react-planner
{'event': 'log',

## Events History

In [1]:
from illufly.io import LocalFileEventsHistory
from illufly.chat import FakeLLM
llm = FakeLLM(events_history=LocalFileEventsHistory("__EVENTS__"))

In [3]:
llm.events_history.load_events_history()

('20241128-08897-0000-6863',
 {'threads': [['FakeLLM.4413207552', '20241128-08903-0000-3182'],
   ['FakeLLM.4519100032', '20241129-45516-0005-5280']],
  'callings': {'20241128-08903-0000-7527': {'id': '20241128-08903-0000-1506',
    'event_type': 'log',
    'data': '{"block_id": "20241128-08903-0000-1506", "block_type": "runnable", "content": "FakeLLM.4413207552", "content_id": "20241128-08903-0001-5465", "created_at": "2024-11-28T23:48:23.412970", "thread_id": null, "calling_id": "20241128-08903-0000-7527", "agent_name": "FakeLLM.4413207552", "model_name": null}'},
   '20241129-45516-0011-9642': {'id': '20241129-45516-0288-6749',
    'event_type': 'log',
    'data': '{"block_id": "20241129-45516-0288-6749", "block_type": "runnable", "content": "FakeLLM.4519100032", "content_id": "20241129-45516-0289-2091", "created_at": "2024-11-29T09:58:36.434904", "thread_id": null, "calling_id": "20241129-45516-0011-9642", "agent_name": "FakeLLM.4519100032", "model_name": null}'}}})

In [16]:
llm.events_history.events_history_id

'20241128-08897-0000-6863'

In [17]:
llm.events_history.store

{'20241128-08897-0000-6863': {'threads': [['FakeLLM.4413207552',
    '20241128-08903-0000-3182']],
  'callings': {'20241128-08903-0000-7527': {'id': '20241128-08903-0000-1506',
    'event_type': 'log',
    'data': '{"block_id": "20241128-08903-0000-1506", "block_type": "runnable", "content": "FakeLLM.4413207552", "content_id": "20241128-08903-0001-5465", "created_at": "2024-11-28T23:48:23.412970", "thread_id": null, "calling_id": "20241128-08903-0000-7527", "agent_name": "FakeLLM.4413207552", "model_name": null}'}}}}

In [18]:
llm("hello")

[AGENT] FakeLLM.4519100032
Reply >> hello


'Reply >> hello'

In [2]:
llm.events_history.load_events_history(-1)

('20241128-08897-0000-6863',
 {'threads': [['FakeLLM.4413207552', '20241128-08903-0000-3182'],
   ['FakeLLM.4519100032', '20241129-45516-0005-5280']],
  'callings': {'20241128-08903-0000-7527': {'id': '20241128-08903-0000-1506',
    'event_type': 'log',
    'data': '{"block_id": "20241128-08903-0000-1506", "block_type": "runnable", "content": "FakeLLM.4413207552", "content_id": "20241128-08903-0001-5465", "created_at": "2024-11-28T23:48:23.412970", "thread_id": null, "calling_id": "20241128-08903-0000-7527", "agent_name": "FakeLLM.4413207552", "model_name": null}'},
   '20241129-45516-0011-9642': {'id': '20241129-45516-0288-6749',
    'event_type': 'log',
    'data': '{"block_id": "20241129-45516-0288-6749", "block_type": "runnable", "content": "FakeLLM.4519100032", "content_id": "20241129-45516-0289-2091", "created_at": "2024-11-29T09:58:36.434904", "thread_id": null, "calling_id": "20241129-45516-0011-9642", "agent_name": "FakeLLM.4519100032", "model_name": null}'}}})